In [1]:
import numpy as np

from scipy import stats

In [2]:
def read_sessions_from_file(file_name):
    sessions = []
    with open(file_name, 'r') as f_name:
        for line in f_name:
            session = map(int, line.split())
            sessions.append(session)
    return sessions

In [3]:
sessions_1 = np.asarray(read_sessions_from_file('c1.txt'))
sessions_1.shape

(1000000, 10)

In [4]:
sessions_2 = np.asarray(read_sessions_from_file('c2.txt'))
sessions_2.shape

(1000000, 10)

In [5]:
def ctr_metric(session):
    session = session.sum(axis=1)
    return float(sum(session > 0)) / len(session)

In [6]:
def click_into_first_result(session):
    return sum(session[:,0] == 1) / float(len(session))

In [7]:
def number_of_rejects(session):
    session = session.sum(axis=1)
    return float(sum(session == 0)) / len(session)

In [8]:
def t_criteria(sample_1, sample_2, alpha=0.05):
    """
    H0: mean_1 == mean_2
    """
    
    n_1, n_2 = float(len(sample_1)), float(len(sample_2))
    
    mean_1, var_1 = np.mean(sample_1), np.var(sample_1, ddof=1)
    mean_2, var_2 = np.mean(sample_2), np.var(sample_2, ddof=1)
        
    var = ((n_1 - 1.0) * var_1 + (n_2 - 1.0) * var_2) / (n_1 + n_2 - 2)
    
    t_stat = np.sqrt(n_1 * n_2 / (n_1 + n_2)) * (mean_1 - mean_2) / np.sqrt(var)
    t_crit = stats.t.ppf(1.0 - 0.5 * alpha, df = n_1 + n_2 - 2)
        
    return t_stat > t_crit

In [9]:
def sample_from(session, frac=0.1):
    length = len(session)
    index = np.random.choice(length, size=int(frac * length)) 
    return session[index]

In [10]:
def sample_indexes_from(session, frac=0.1):
    length = len(session)
    return np.random.choice(length, size=int(frac * length)) 

In [11]:
def split_into_groups(sample, size=1000):
    return [sample[b:b+size] for b in range(0, len(sample), size)]

In [12]:
def bootstrap(metric=ctr_metric, n_steps=1000):
    print 'Bootstrap starts...'
    
    count = 0
    
    for step in range(n_steps):
        # index = sample_indexes_from(sessions_1, frac=0.01)
        # groups_1 = split_into_groups(sessions_1[index])
        # groups_2 = split_into_groups(sessions_2[index])
        
        groups_1 = split_into_groups(sample_from(sessions_1, frac=0.01))
        groups_2 = split_into_groups(sample_from(sessions_2, frac=0.01))
        
        metric_1 = map(metric, groups_1)
        metric_2 = map(metric, groups_2)
        
        count += int(t_criteria(metric_1, metric_2, alpha=0.05))
        
        if (step + 1) % 10 == 0:
            print '\r{:>4} of {} iters completed...'.format(step+1, n_steps),
    
    print '\nBootstrap finished!'
    
    return float(count) / n_steps

In [13]:
bootstrap(ctr_metric, 1000)

Bootstrap starts...
1000 of 1000 iters completed... 
Bootstrap finished!


0.987

In [14]:
bootstrap(click_into_first_result, 1000)

Bootstrap starts...
1000 of 1000 iters completed... 
Bootstrap finished!


1.0

Среднее число кликов в 1 результат в **session_1** больше, чем в **session_2**, поэтому первая сессия лучше.

In [15]:
bootstrap(number_of_rejects, 1000)

Bootstrap starts...
1000 of 1000 iters completed... 
Bootstrap finished!


0.0

Среднее число отказов в **session_1** не превышает число отказов в **session_2**, поэтому первая сессия лучше.